### Importing Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
from scipy.stats import norm

yf.pdr_override()

### Black-Scholes Model

In [2]:
def d1 (stock_price, strike_price, riskfree_rate, stdev, time_horizon_years):
    return (np.log(stock_price / strike_price) + (riskfree_rate + stdev ** 2 / 2) * time_horizon_years) / (stdev * np.sqrt(time_horizon_years))

def d2 (stock_price1, strike_price, riskfree_rate, stdev, time_horizon_years):
    return (np.log(stock_price / strike_price) + (riskfree_rate - stdev ** 2 / 2) * time_horizon_years) / (stdev * np.sqrt(time_horizon_years))

def black_scholes (stock_price, strike_price, riskfree_rate, stdev, time_horizon_years):
    return (stock_price * norm.cdf(d1(stock_price, strike_price, riskfree_rate, stdev, time_horizon_years))) - (strike_price * np.exp(-riskfree_rate * time_horizon_years) * norm.cdf(d2(stock_price, strike_price, riskfree_rate, stdev, time_horizon_years)))

ticker = 'MSFT'
data = pd.DataFrame()
data[ticker] = pdr.get_data_yahoo(ticker, start='2017-1-1', end='2022-12-31')['Adj Close']

stock_price = data.iloc[-1]
stock_price

[*********************100%***********************]  1 of 1 completed


MSFT    238.69902
Name: 2022-12-30 00:00:00, dtype: float64

In [3]:
riskfree_rate = 0.025
strike_price = 250.0
trading_days = 252
time_horizon_years = 2

log_returns = np.log(1 + data.pct_change())
stdev = log_returns.std() * np.sqrt(trading_days)

'Black-Scholes Price: ' + black_scholes (stock_price, strike_price, riskfree_rate, stdev, time_horizon_years).MSFT.astype(str)

'Black-Scholes Price: 39.13650590505772'

### Euler Discretization

In [12]:
stdev_array = stdev.values
delta_t = time_horizon_years / trading_days
iterations = 1000

z = np.random.standard_normal((trading_days + 1, iterations))
stock_price = np.zeros_like(z)
stock_price[0] = data.iloc[-1]

for time in range (1, trading_days + 1):
    stock_price[time] = stock_price[time - 1] * np.exp((riskfree_rate - 0.5 * stdev_array ** 2) * delta_t + stdev_array * np.sqrt(delta_t) * z[time])

euler = np.exp(-riskfree_rate * time_horizon_years) * np.sum(np.maximum(stock_price[-1] - strike_price, 0)) / iterations
'Euler Price: ' + euler.astype(str)

'Euler Price: 39.67743366195735'